In [1]:
import pandas as pd
import numpy as np


# Amazon stock 10 year analysis


In [2]:

df=pd.read_csv("AMZN.csv")

nasdaq=pd.read_csv("NASDAQ.csv")
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-01-03,181.369995,186.000000,181.210007,184.220001,184.220001,5331400
1,2011-01-04,186.149994,187.699997,183.779999,185.009995,185.009995,5031800
2,2011-01-05,184.100006,187.449997,184.070007,187.419998,187.419998,3418800
3,2011-01-06,186.500000,187.410004,185.250000,185.860001,185.860001,3179700
4,2011-01-07,187.880005,188.449997,183.740005,185.490005,185.490005,5221700
...,...,...,...,...,...,...,...
2258,2019-12-23,1788.260010,1793.000000,1784.510010,1793.000000,1793.000000,2136400
2259,2019-12-24,1793.810059,1795.569946,1787.579956,1789.209961,1789.209961,881300
2260,2019-12-26,1801.010010,1870.459961,1799.500000,1868.770020,1868.770020,6005400
2261,2019-12-27,1882.920044,1901.400024,1866.010010,1869.800049,1869.800049,6186600


In [3]:
df.dropna(inplace=True)

In [4]:
df['Returns_n']=0
df['Returns_d']=0
df['Spread_p']=0
for i in range(1,len(df)):
    df.iloc[i,df.columns.get_loc("Returns_n")]=df.iloc[i]['Open']-df.iloc[i-1]['Close'] #Returns_n columns gives return when a stock is bought on previous day night and sold on todays morning
    df.iloc[i,df.columns.get_loc("Returns_d")]=df.iloc[i]['Close']-df.iloc[i]['Open']#Returns_d shows daily return of stock
    df.iloc[i,df.columns.get_loc("Spread_p")]=((df.iloc[i]['High']-df.iloc[i]['Low'])/df.iloc[i]['Low'] )*100 #stock high-low
df

,Date,Open,High,Low,Close,Adj Close,Volume,Returns_n,Returns_d,Spread_p
0,2011-01-03,181.369995,186.000000,181.210007,184.220001,184.220001,5331400,0.000000,0.000000,0.000000
1,2011-01-04,186.149994,187.699997,183.779999,185.009995,185.009995,5031800,1.929993,-1.139999,2.132984
2,2011-01-05,184.100006,187.449997,184.070007,187.419998,187.419998,3418800,-0.909989,3.319992,1.836252
3,2011-01-06,186.500000,187.410004,185.250000,185.860001,185.860001,3179700,-0.919998,-0.639999,1.165994
4,2011-01-07,187.880005,188.449997,183.740005,185.490005,185.490005,5221700,2.020004,-2.390000,2.563400
...,...,...,...,...,...,...,...,...,...,...
2258,2019-12-23,1788.260010,1793.000000,1784.510010,1793.000000,1793.000000,2136400,1.760010,4.739990,0.475760
2259,2019-12-24,1793.810059,1795.569946,1787.579956,1789.209961,1789.209961,881300,0.810059,-4.600098,0.446972
2260,2019-12-26,1801.010010,1870.459961,1799.500000,1868.770020,1868.770020,6005400,11.800049,67.760010,3.943315
2261,2019-12-27,1882.920044,1901.400024,1866.010010,1869.800049,1869.800049,6186600,14.150024,-13.119995,1.896561


In [5]:
nasdaq.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume
0,03/01/2011,2676.649902,2704.860107,2676.340088,2691.52002,2691.52002,1919660000
1,04/01/2011,2699.860107,2700.879883,2663.639893,2681.25000,2681.25000,2015440000


### Theory 1

In [6]:
#Creating a port dataframe in which if we invest 100 rupes at the start what returns it gives after 9 years and comparing it with NASDAQ index
#investing at closing price only if the daily return for that day is negative
port=pd.DataFrame(df,columns=['Inv'])
port.iloc[0,port.columns.get_loc("Inv")]=100

for i in range(1,len(df)-1):
    if df.iloc[i]['Returns_d'] < 0 :
        port.iloc[i,port.columns.get_loc("Inv")]=df.iloc[i+1]['Returns_n']*port.iloc[i-1]['Inv']/df.iloc[i-1]['Close']+port.iloc[i-1]['Inv']
    else:
        port.iloc[i,port.columns.get_loc("Inv")]=port.iloc[i-1]['Inv']

    
        
#calculating daily return on index NASDAQ

p=pd.DataFrame(nasdaq,columns=['Date','NASDAQ'])
p.iloc[0,p.columns.get_loc("NASDAQ")]=100
for i in range(1,len(nasdaq)):
    p.iloc[i,p.columns.get_loc("NASDAQ")]= ((nasdaq.iloc[i]['Close']-nasdaq.iloc[i]['Open'])*p.iloc[i-1]['NASDAQ']/nasdaq.iloc[i]['Open'])+p.iloc[i-1]['NASDAQ']

final=pd.concat([port,p] ,axis=1,ignore_index=False)
final.set_index(['Date'],inplace=True)
final.dropna(inplace=True)
final

,Inv,NASDAQ
Date,,
03/01/2011,100.000000,100.000000
04/01/2011,99.506031,99.310701
05/01/2011,99.506031,100.361411
06/01/2011,100.578503,100.565891
07/01/2011,100.334978,100.210743
...,...,...
20/12/2019,430.591516,152.841205
23/12/2019,430.591516,152.763509
24/12/2019,433.425315,152.727175


### Theory 2

In [7]:
#Creating a port dataframe in which if we invest 100 rupes at the start what returns it gives after 9 years and comparing it with NASDAQ index
#Investing at closing price and selling at opening next day
port2=pd.DataFrame(df,columns=['Inv'])
port2.iloc[0,port.columns.get_loc("Inv")]=100

for i in range(1,len(df)-1):
    port2.iloc[i,port2.columns.get_loc("Inv")]=df.iloc[i+1]['Returns_n']*port2.iloc[i-1]['Inv']/df.iloc[i-1]['Close']+port2.iloc[i-1]['Inv']
    
    
        
#calculating daily return on index NASDAQ

p1=pd.DataFrame(nasdaq,columns=['Date','NASDAQ'])
p1.iloc[0,p1.columns.get_loc("NASDAQ")]=100
for i in range(1,len(nasdaq)):
    p1.iloc[i,p1.columns.get_loc("NASDAQ")]= ((nasdaq.iloc[i]['Close']-nasdaq.iloc[i]['Open'])*p1.iloc[i-1]['NASDAQ']/nasdaq.iloc[i]['Open'])+p1.iloc[i-1]['NASDAQ']

finale=pd.concat([port2,p1] ,axis=1,ignore_index=False)
finale.set_index(['Date'],inplace=True)
finale.dropna(inplace=True)
finale

,Inv,NASDAQ
Date,,
03/01/2011,100.000000,100.000000
04/01/2011,99.506031,99.310701
05/01/2011,99.011218,100.361411
06/01/2011,100.078357,100.565891
07/01/2011,99.836043,100.210743
...,...,...
20/12/2019,904.048023,152.841205
23/12/2019,904.457949,152.763509
24/12/2019,910.410346,152.727175


Though the number of transactions are increased in second one and it gives a higher return, the brokerage cost is also greatly increased(which is not taken into consideration here)

## Comaprison of change in monthly stock prices due to day and night respectively

In [8]:
df['Date']= pd.to_datetime(df['Date'])
df.set_index("Date",inplace=True)


In [9]:
df1=pd.DataFrame(columns=['Month','Year'])

df1['Returns_n']=df.Returns_n.resample('M').sum()
df1['Returns_d']=df.Returns_d.resample('M').sum()
df1['Close']=df.Close.resample('M').mean()
df1['Open']=df.Open.resample('M').mean()
df1.reset_index(inplace=True)
df1['Month']=df1.Date.dt.month
df1['Year']=df1.Date.dt.year

df1

,Date,Month,Year,Returns_n,Returns_d,Close,Open
0,2011-01-31,1,2011,-9.960005,-4.619997,182.352499,182.440999
1,2011-02-28,2,2011,-7.530000,11.179994,181.117894,180.529473
2,2011-03-31,3,2011,2.849993,3.990019,168.803914,168.630435
3,2011-04-30,4,2011,2.420001,13.259992,184.576500,183.913500
4,2011-05-31,5,2011,-2.879992,3.759996,198.166667,197.987620
...,...,...,...,...,...,...,...
103,2019-08-31,8,2019,36.439942,-126.929932,1793.602722,1799.372264
104,2019-09-30,9,2019,11.520019,-51.900024,1799.120996,1801.715997
105,2019-10-31,10,2019,-32.709959,73.459959,1752.331750,1749.137839
106,2019-11-30,11,2019,45.020511,-20.880496,1774.293994,1775.338019


In [10]:
#Considering that the stock price was intially 100 and calulating the change in stock price from Feb 2011 to Dec 2019.
#Just to get an idea whether the stock price increased during thre day or night.

df2=pd.DataFrame(df1,columns=['Month','Year'])
df2['Price_n']=0
df2['Price_d']=0
df2.iloc[0,df2.columns.get_loc("Price_n")]=100
df2.iloc[0,df2.columns.get_loc("Price_d")]=100
for i in range(1,108):
   
    df2.iloc[i,df2.columns.get_loc("Price_n")]=df1.iloc[i]['Returns_n']*df2.iloc[i-1]['Price_n']/df1.iloc[i-1]['Close']+df2.iloc[i-1]['Price_n']
    df2.iloc[i,df2.columns.get_loc("Price_d")]=df1.iloc[i]['Returns_d']*df2.iloc[i-1]['Price_d']/df1.iloc[i]['Open']+df2.iloc[i-1]['Price_d']
    
    
df2

,Month,Year,Price_n,Price_d
0,1,2011,100.000000,100.000000
1,2,2011,95.870635,106.192891
2,3,2011,97.379214,108.705555
3,4,2011,98.775259,116.543125
4,5,2011,97.234044,118.756403
...,...,...,...,...
103,8,2019,936.987839,83.907581
104,9,2019,943.005960,81.490548
105,10,2019,925.861095,84.912973
106,11,2019,949.648109,83.914276


Thus it can be safely assumed that the rise in amazon stock price from 181 to 1874  is all during the night while there is comparatively no increase during the day. The rise in stock price mostly due to night is an important conclusion.
